# Case Study 1 - Data Modality

## The Task
Get used to loading datasets with the library and generating synthetic data from them, whatever the modality of the real data.

### Imports
Lets get the imports out of the way. We import the required standard and 3rd party libraries and relevant Synthcity modules. We can also set the level of logging here, using Synthcity's bespoke logger. 

In [ ]:
# Standard
import sys
import warnings
from pathlib import Path

# 3rd party
import numpy as np
import pandas as pd

# synthcity
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import (GenericDataLoader, SurvivalAnalysisDataLoader, TimeSeriesDataLoader,TimeSeriesSurvivalDataLoader)

# Configure warnings and logging
warnings.filterwarnings("ignore")

# Set the level for the logging
# log.add(sink=sys.stderr, level="DEBUG")
log.remove()

## Loading data of different modalities
### Static Data
Now  we will start with the simplest example, static tabular data. First we need to load the dataset.

In [ ]:
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True, as_frame=True)
X["target"] = y
display(X)


Then we pass it to the `GenericDataLoader` object.

In [ ]:
loader = GenericDataLoader(
    X,
    target_column="target",
    sensitive_columns=["sex"],
)

We can print out different methods that are compatible with our data by calling `Plugins().list()` with a relevant list passed to the categories parameter.

In [ ]:
print(Plugins(categories=["generic"]).list())

No need to worry about the code in this next block here, we will go into lots of detail in how to generate synthetic data in the case studies to come. It is here purely to demonstrate that our dataset can be used to generate synthetic data using the synthcity module.

In [ ]:
syn_model = Plugins().get("marginal_distributions")
syn_model.fit(loader)
syn_model.generate(count=10).dataframe()

## Static survival
Next lets look at censored data. Censoring is a form of missing data problem in which time to event is not observed for reasons such as termination of study before all recruited subjects have shown the event of interest or the subject has left the study prior to experiencing an event. Censoring is common in survival analysis. 

For our next example we will load a static survival dataset.

In [ ]:
from sksurv.datasets import load_veterans_lung_cancer

data_x, data_y = load_veterans_lung_cancer()
data_x["status"], data_x["survival_in_days"] = [record[0] for record in data_y], [record[1] for record in data_y]

loader = SurvivalAnalysisDataLoader(
    data_x,
    target_column="status",
    time_to_event_column="survival_in_days",
)
print(loader.info())

syn_model = Plugins().get("marginal_distributions", n_iter=1)
syn_model.fit(loader)
syn_model.generate(count=10)
display(syn_model.generate(count=10).dataframe())


### Regular Time Series

In this next example we will load up a simple regular time series dataset and show that it is compatible with Synthcity. The temporal data must be passed to the loader as a list of dataframes, where each dataframe in the list refers to a different record and contains all time points for the record. So, there is a small amount of pre-processing to get our data into the right shape. As the time series in regular we can simply pass a sequential list for each record.

In [ ]:
from sktime.datasets import load_basic_motions

X, y = load_basic_motions(
    split="TRAIN", return_X_y=True, return_type="pd-multiindex"
)
num_instances = len(set((x[0] for x in X.index)))
num_time_steps = len(set((x[1] for x in X.index)))

temporal_data = [X.loc[i] for i in range(num_instances)]
y = df = pd.DataFrame(y, columns=["label"])
observation_times = [range(num_time_steps) for i in range(num_instances)]

loader = TimeSeriesDataLoader(
    temporal_data=temporal_data,
    observation_times=observation_times,
    outcome=y,
)

print(loader.info())

syn_model = Plugins().get("marginal_distributions", n_iter=1)
syn_model.fit(loader)
syn_model.generate(count=10)
display(syn_model.generate(count=10).dataframe())


### Irregular Time Series

Now lets load a time series dataset and show that that is also compatible with Synthcity.

In [ ]:
import numpy as np
from synthcity.utils.datasets.time_series.google_stocks import GoogleStocksDataloader

static_data, temporal_data, horizons, outcome = GoogleStocksDataloader().load()

loader = TimeSeriesDataLoader(
    temporal_data=temporal_data,
    observation_times=horizons,
    static_data=static_data,
    outcome=outcome,
)
print(loader.info())
display(loader.dataframe())

syn_model = Plugins().get("marginal_distributions", n_iter=1)
syn_model.fit(loader)
syn_model.generate(count=10)
display(syn_model.generate(count=10).dataframe())

### Composite Irregular Time Series Survival Analysis

In this final example we will look at composite data while adding all the other more complex formats we have looked at so far. This next dataset is a composite irregular time series survival analysis dataset. Even complex datasets such as this are compatible with Synthcity. We can load this data using the `TimeSeriesSurvivalDataLoader`. Then by calling `loader.info()`, we can check the information about the dataset. It contains both one static feature ("sex") and 14 temporal features, making it a composite dataset. The `seq_time_id` field shows the irregular time sampling, which we create by passing the values to the `observation_times` parameter of the `TimeSeriesSurvivalDataLoader` object. And finally, we are formulating this data as a survival analysis problem, which is indicated by the presence of a `time_to_event` field.

In [ ]:
import numpy as np
from synthcity.utils.datasets.time_series.pbc import PBCDataloader

(
    static_surv,
    temporal_surv,
    temporal_surv_horizons,
    outcome_surv,
) = PBCDataloader().load()
T, E = outcome_surv

horizons = [0.25, 0.5, 0.75]
time_horizons = np.quantile(T, horizons).tolist()

loader = TimeSeriesSurvivalDataLoader(
    temporal_data=temporal_surv,
    observation_times=temporal_surv_horizons,
    static_data=static_surv,
    T=T,
    E=E,
    time_horizons=time_horizons,
)

print(loader.info())

syn_model = Plugins().get("marginal_distributions", n_iter=1)
syn_model.fit(loader)
syn_model.generate(count=10)
display(syn_model.generate(count=10).dataframe())

### Create synthetic datasets
 - Above we have generated data with the debugging method `"marginal_distributions"`. Now, using `Plugins().list()` or the documentation find another method that is compatible with some of the datasets to see if you can generate your own snthetic data.